# Why doesnt zarr append work?


Load the first dat file

In [2]:
%%time 
import sys
sys.path.append("/home/jovyan/XApRES/src/XApRES/")
import ApRESDefs
 
xa = ApRESDefs.xapres(max_range=1400)
xa.load_all(directory='gs://ldeo-glaciology/GL_apres_2022/A101', 
            remote_load = True,
            file_numbers_to_process=[3]#range(3)
           )
xa.data

100%|██████████| 94/94 [02:13<00:00,  1.42s/it]


CPU times: user 1min 47s, sys: 11.8 s, total: 1min 59s
Wall time: 2min 17s


<xarray.Dataset>
Dimensions:          (time: 94, chirp_time: 40001, chirp_num: 20,
                      attenuator_setting_pair: 2, profile_range: 6658)
Coordinates:
  * time             (time) datetime64[ns] 2022-05-27T15:06:30 ... 2022-05-28...
  * chirp_time       (chirp_time) float64 0.0 2.5e-05 5e-05 ... 1.0 1.0 1.0
  * profile_range    (profile_range) float64 0.0 0.2103 ... 1.4e+03 1.4e+03
  * chirp_num        (chirp_num) int64 0 1 2 3 4 5 6 7 ... 13 14 15 16 17 18 19
    filename         (time) <U83 'ldeo-glaciology/GL_apres_2022/A101/CardA/DI...
    burst_number     (time) int64 0 1 2 3 4 5 6 7 8 ... 86 87 88 89 90 91 92 93
    AFGain           (attenuator_setting_pair) int64 -4 -14
    attenuator       (attenuator_setting_pair) float64 5.0 5.0
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 ...
    profile          (time, profile_range, chirp_num, attenuator_setting_pair) complex128 ...
    latitude         (time) float64 68.71 68.71 68.71 ... 68.71 68.71 68.71
    longitude        (time) float64 -49.55 -49.55 -49.55 ... -49.55 -49.55
    battery_voltage  (time) float64 13.61 13.56 13.56 ... 13.75 13.74 13.74
    temperature_1    (time) float64 8.5 7.32 5.359 5.75 ... 510.9 508.9 506.8
    temperature_2    (time) float64 9.125 6.766 4.023 ... 4.023 4.023 3.625
Attributes:
    time created:  2022-11-16 01:43:58.577605

In [3]:
xa.data.nbytes/1e9

1.604185296

Get a mapper to a new zarr store

In [4]:
import os
import gcsfs
import fsspec
import json

In [5]:
with open('../../secrets/ldeo-glaciology-bc97b12df06b.json') as token_file:
    token = json.load(token_file)

filename = 'gs://ldeo-glaciology/GL_apres_2022_zarr_test_02'
    
mapper = fsspec.get_mapper(filename, mode='w', token=token)



Write to the zarr

In [6]:
ds = xa.data
ds.attrs = []
ds.to_zarr(mapper, mode='w');

Reload the zarr to check it exists

In [9]:
import xarray as xr
ds_reloaded = xr.open_dataset(filename, engine='zarr', consolidated=False, chunks = "auto")

ds_reloaded

<xarray.Dataset>
Dimensions:          (attenuator_setting_pair: 2, time: 94, chirp_time: 40001,
                      chirp_num: 20, profile_range: 6658)
Coordinates:
    AFGain           (attenuator_setting_pair) int64 dask.array<chunksize=(2,), meta=np.ndarray>
    attenuator       (attenuator_setting_pair) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    burst_number     (time) int64 dask.array<chunksize=(94,), meta=np.ndarray>
  * chirp_num        (chirp_num) int64 0 1 2 3 4 5 6 7 ... 13 14 15 16 17 18 19
  * chirp_time       (chirp_time) timedelta64[ns] 00:00:00 ... 00:00:01
    filename         (time) <U83 dask.array<chunksize=(94,), meta=np.ndarray>
  * profile_range    (profile_range) float64 0.0 0.2103 ... 1.4e+03 1.4e+03
  * time             (time) datetime64[ns] 2022-05-27T15:06:30 ... 2022-05-28...
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    battery_voltage  (time) float64 dask.array<chunksize=(94,), meta=np.ndarray>
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 dask.array<chunksize=(36, 15003, 15, 2), meta=np.ndarray>
    latitude         (time) float64 dask.array<chunksize=(94,), meta=np.ndarray>
    longitude        (time) float64 dask.array<chunksize=(94,), meta=np.ndarray>
    profile          (time, profile_range, chirp_num, attenuator_setting_pair) complex128 dask.array<chunksize=(36, 4995, 15, 2), meta=np.ndarray>
    temperature_1    (time) float64 dask.array<chunksize=(94,), meta=np.ndarray>
    temperature_2    (time) float64 dask.array<chunksize=(94,), meta=np.ndarray>

Load the next dat file

In [7]:
xa = ApRESDefs.xapres(max_range=1400)
xa.load_all(directory='gs://ldeo-glaciology/GL_apres_2022/A101', 
            remote_load = True,
            file_numbers_to_process=range(1,2)
           )
xa.data

100%|██████████| 1/1 [00:01<00:00,  1.30s/it]


<xarray.Dataset>
Dimensions:          (time: 1, chirp_time: 40001, chirp_num: 20,
                      attenuator_setting_pair: 2, profile_range: 6658)
Coordinates:
  * time             (time) datetime64[ns] 2022-05-26T15:31:02
  * chirp_time       (chirp_time) float64 0.0 2.5e-05 5e-05 ... 1.0 1.0 1.0
  * profile_range    (profile_range) float64 0.0 0.2103 ... 1.4e+03 1.4e+03
  * chirp_num        (chirp_num) int64 0 1 2 3 4 5 6 7 ... 13 14 15 16 17 18 19
    filename         (time) <U83 'ldeo-glaciology/GL_apres_2022/A101/CardA/DI...
    burst_number     (time) int64 0
    AFGain           (attenuator_setting_pair) int64 -4 -14
    attenuator       (attenuator_setting_pair) float64 5.0 5.0
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 ...
    profile          (time, profile_range, chirp_num, attenuator_setting_pair) complex128 ...
    latitude         (time) float64 68.71
    longitude        (time) float64 -49.55
    battery_voltage  (time) float64 13.93
    temperature_1    (time) float64 2.812
    temperature_2    (time) float64 2.25
Attributes:
    time created:  2022-11-15 22:14:55.640130

Try to append to the zarr

In [8]:
ds = xa.data
ds.attrs = []
ds.to_zarr(mapper, mode='a', append_dim="time")

When we reload the zarr it only has 13 time steps still, not 14. Why?

In [10]:
ds_reloaded = xr.open_dataset(filename, engine='zarr', consolidated=False)
ds_reloaded

<xarray.Dataset>
Dimensions:          (attenuator_setting_pair: 2, time: 13, chirp_time: 40001,
                      chirp_num: 20, profile_range: 6658)
Coordinates:
    AFGain           (attenuator_setting_pair) int64 ...
    attenuator       (attenuator_setting_pair) float64 ...
    burst_number     (time) int64 ...
  * chirp_num        (chirp_num) int64 0 1 2 3 4 5 6 7 ... 13 14 15 16 17 18 19
  * chirp_time       (chirp_time) timedelta64[ns] 00:00:00 ... 00:00:01
    filename         (time) <U83 ...
  * profile_range    (profile_range) float64 0.0 0.2103 ... 1.4e+03 1.4e+03
  * time             (time) datetime64[ns] 2022-05-22T19:40:20 ... 2022-05-22...
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    battery_voltage  (time) float64 ...
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 ...
    latitude         (time) float64 ...
    longitude        (time) float64 ...
    profile          (time, profile_range, chirp_num, attenuator_setting_pair) complex128 ...
    temperature_1    (time) float64 ...
    temperature_2    (time) float64 ...